In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


pd.set_option("display.max_columns",200)

def print_value_counts(df):
    for i in df.columns:
        print(f'column: {i}')
        display(df[i].value_counts())
        print()

Steps:

1. Select relevant cols
2. Impute NA - strategies: categorical - mode and NA, numeric median (or sth else)
3. one-hot encode categorical
4. merge categorical and numeric 
5. Fit models

Contents:

- load data
- train test split
- preprocess categorical vars
- preprocess numeric vars
- run CV

## Load data

In [2]:
master_table = pd.read_pickle('data/master_table.pkl')

In [3]:
master_table.head()

,accident_id,lighting,localization,intersection_type,weather,collision_type,com,address,gps,lat,long,departament,time,year,month,hour,road_category,road_regime,no_lanes,reserved_lane,road_gradient,road_plan,road_condition,infrastructure,accident_situation,user_id,place_in_car,user_type,injury_type,sex,equipment_used,pedestrian_action,pedestrian_alone,year_of_birth,vechicle_number,y
0,200500000001,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,11,CD41B,Métropole,5051500,294400,590,2005-01-05 19:00:00,2005.0,1.0,19.0,Departmental Road,Bidirectional,2.0,NaN,Dish,Straight part,normal,NaN,On the road,1678507,1,Driver,Light injury,Male,yes,not specified or not applicable,NaN,1976,A01,0
1,200500000001,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,11,CD41B,Métropole,5051500,294400,590,2005-01-05 19:00:00,2005.0,1.0,19.0,Departmental Road,Bidirectional,2.0,NaN,Dish,Straight part,normal,NaN,On the road,1678508,1,Driver,Hospitalized wounded,Female,yes,not specified or not applicable,NaN,1968,B02,1
2,200500000001,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,11,CD41B,Métropole,5051500,294400,590,2005-01-05 19:00:00,2005.0,1.0,19.0,Departmental Road,Bidirectional,2.0,NaN,Dish,Straight part,normal,NaN,On the road,1678509,2,Passenger,Unscathed,Male,yes,not specified or not applicable,NaN,1964,B02,0
3,200500000001,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,11,CD41B,Métropole,5051500,294400,590,2005-01-05 19:00:00,2005.0,1.0,19.0,Departmental Road,Bidirectional,2.0,NaN,Dish,Straight part,normal,NaN,On the road,1678510,4,Passenger,Unscathed,Male,yes,not specified or not applicable,NaN,2004,B02,0
4,200500000001,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,11,CD41B,Métropole,5051500,294400,590,2005-01-05 19:00:00,2005.0,1.0,19.0,Departmental Road,Bidirectional,2.0,NaN,Dish,Straight part,normal,NaN,On the road,1678511,5,Passenger,Unscathed,Male,yes,not specified or not applicable,NaN,1998,B02,0


## EDA with filtering approach and feature selection

In [4]:
master_table['year_of_birth'] = master_table['year_of_birth'].astype(float)

In [5]:
cols_to_model = [
#  'accident_id',
 'lighting',
 'localization',
 'intersection_type',
 'weather',
 'collision_type',
#  'com',
#  'address',
#  'gps',
#  'lat',
#  'long',
#  'departament',
#  'time',
 'year',
 'month',
 'hour',
 'road_category',
 'road_regime',
 'no_lanes',
 'reserved_lane',
 'road_gradient',
 'road_plan',
 'road_condition',
 'infrastructure',
 'accident_situation',
#  'user_id',
 'place_in_car',
 'user_type',
#  'injury_type',
 'sex',
 'equipment_used',
 'pedestrian_action',
 'pedestrian_alone',
 'year_of_birth',
#  'vechicle_number',
#  'y'
]


In [6]:
master_table[cols_to_model].dtypes

lighting               object
localization           object
intersection_type      object
weather                object
collision_type         object
year                  float64
month                 float64
hour                  float64
road_category          object
road_regime            object
no_lanes              float64
reserved_lane          object
road_gradient          object
road_plan              object
road_condition         object
infrastructure         object
accident_situation     object
place_in_car           object
user_type              object
sex                    object
equipment_used         object
pedestrian_action      object
pedestrian_alone       object
year_of_birth         float64
dtype: object

TODO:

- get dummies (with na)
- 0 variance
- m

### Categorical columns

In [7]:
df_cat = master_table[cols_to_model].select_dtypes('object')
df_cat.head()

,lighting,localization,intersection_type,weather,collision_type,road_category,road_regime,reserved_lane,road_gradient,road_plan,road_condition,infrastructure,accident_situation,place_in_car,user_type,sex,equipment_used,pedestrian_action,pedestrian_alone
0,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NaN,Dish,Straight part,normal,NaN,On the road,1,Driver,Male,yes,not specified or not applicable,NaN
1,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NaN,Dish,Straight part,normal,NaN,On the road,1,Driver,Female,yes,not specified or not applicable,NaN
2,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NaN,Dish,Straight part,normal,NaN,On the road,2,Passenger,Male,yes,not specified or not applicable,NaN
3,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NaN,Dish,Straight part,normal,NaN,On the road,4,Passenger,Male,yes,not specified or not applicable,NaN
4,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NaN,Dish,Straight part,normal,NaN,On the road,5,Passenger,Male,yes,not specified or not applicable,NaN


Replacing NA's - for now with 'NA'

In [8]:
si = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 'NA', add_indicator=False)
temp = si.fit_transform(df_cat)
df_cat2 = pd.DataFrame(temp, columns = df_cat.columns)
df_cat2.head()

,lighting,localization,intersection_type,weather,collision_type,road_category,road_regime,reserved_lane,road_gradient,road_plan,road_condition,infrastructure,accident_situation,place_in_car,user_type,sex,equipment_used,pedestrian_action,pedestrian_alone
0,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NA,Dish,Straight part,normal,NA,On the road,1,Driver,Male,yes,not specified or not applicable,NA
1,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NA,Dish,Straight part,normal,NA,On the road,1,Driver,Female,yes,not specified or not applicable,NA
2,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NA,Dish,Straight part,normal,NA,On the road,2,Passenger,Male,yes,not specified or not applicable,NA
3,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NA,Dish,Straight part,normal,NA,On the road,4,Passenger,Male,yes,not specified or not applicable,NA
4,Night without public lighting,In built-up areas,Out of intersection,Normal,Two vehicles - by the side,Departmental Road,Bidirectional,NA,Dish,Straight part,normal,NA,On the road,5,Passenger,Male,yes,not specified or not applicable,NA


One hot encoding

In [10]:
df_cat3 = pd.get_dummies(df_cat2, drop_first = True)

In [11]:
df_cat3.columns = ['d_' + col for col in list(df_cat3.columns)]

In [12]:
df_cat_out = df_cat3.copy()

In [13]:
df_cat_out.head()

,d_lighting_Night with public lighting not lit,d_lighting_Night with public lighting on,d_lighting_Night without public lighting,d_lighting_Twilight or dawn,d_localization_Out of agglomeration,d_intersection_type_Intersection in T,d_intersection_type_Intersection in X,d_intersection_type_Intersection in Y,d_intersection_type_Intersection with more than 4 branches,d_intersection_type_Level crossing,d_intersection_type_NA,d_intersection_type_Other intersection,d_intersection_type_Out of intersection,d_intersection_type_Place,d_weather_Dazzling weather,d_weather_Fog - smoke,d_weather_Heavy rain,d_weather_Light rain,d_weather_NA,d_weather_Normal,d_weather_Other,d_weather_Snow - hail,d_weather_Strong wind - storm,d_collision_type_Other collision,d_collision_type_Three or more vehicles - multiple collisions,d_collision_type_Three vehicles and more - in chain,d_collision_type_Two vehicles - by the side,d_collision_type_Two vehicles - from the rear,d_collision_type_Two vehicles - frontal,d_collision_type_Without collision,d_road_category_Departmental Road,d_road_category_Highway,d_road_category_NA,d_road_category_National Road,d_road_category_Off public network,d_road_category_Parking lot open to public traffic,d_road_category_other,d_road_regime_NA,d_road_regime_One way,d_road_regime_Separated carriageways,d_road_regime_With variable assignment channels,d_reserved_lane_Cycle Bank,d_reserved_lane_NA,d_reserved_lane_Reserved channel,d_road_gradient_Hill bottom,d_road_gradient_Hilltop,d_road_gradient_NA,d_road_gradient_Slope,d_road_plan_Curved right,"d_road_plan_In ""S""",d_road_plan_NA,d_road_plan_Straight part,d_road_condition_fat - oil,d_road_condition_flooded,d_road_condition_icy,d_road_condition_mud,d_road_condition_normal,d_road_condition_other,d_road_condition_puddles,d_road_condition_snow,d_road_condition_wet,d_infrastructure_Carrefour arranged,d_infrastructure_Exchanger or connection brace,d_infrastructure_NA,d_infrastructure_Pedestrian area,d_infrastructure_Railway,d_infrastructure_Toll zone,d_infrastructure_Underground - tunnel,d_accident_situation_On bike path,d_accident_situation_On emergency stop band,d_accident_situation_On the road,d_accident_situation_On the sidewalk,d_accident_situation_On the verge,d_place_in_car_1,d_place_in_car_2,d_place_in_car_3,d_place_in_car_4,d_place_in_car_5,d_place_in_car_6,d_place_in_car_7,d_place_in_car_8,d_place_in_car_9,d_place_in_car_NA,d_user_type_Passenger,d_user_type_Pedestrian,d_user_type_Pedestrian in rollerblade or scooter,d_sex_Male,d_equipment_used_no,d_equipment_used_yes,d_pedestrian_action_Masked,d_pedestrian_action_Meaning bumping vehicle,d_pedestrian_action_NA,d_pedestrian_action_Opposite direction of the vehicle,d_pedestrian_action_Other,d_pedestrian_action_Playing - running,d_pedestrian_action_With animal,d_pedestrian_action_not specified or not applicable,d_pedestrian_alone_In a group,d_pedestrian_alone_NA,d_pedestrian_alone_Only
0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0


In [14]:
df_cat_out.shape

(1876005, 100)

### Numeric columns 

In [15]:
cols_num = [
 'year',
 'month',
 'hour',
 'year_of_birth'
]

In [16]:
df_num = master_table[cols_num]
df_num.head()

,year,month,hour,year_of_birth
0,2005.0,1.0,19.0,1976.0
1,2005.0,1.0,19.0,1968.0
2,2005.0,1.0,19.0,1964.0
3,2005.0,1.0,19.0,2004.0
4,2005.0,1.0,19.0,1998.0


In [17]:
df_num.isna().mean()

year             0.016829
month            0.016829
hour             0.016829
year_of_birth    0.001253
dtype: float64

Imputing median

In [19]:
si = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=False)
temp = si.fit_transform(df_num)
df_num2 = pd.DataFrame(temp, columns = df_num.columns)
df_num2.head()

,year,month,hour,year_of_birth
0,2005.0,1.0,19.0,1976.0
1,2005.0,1.0,19.0,1968.0
2,2005.0,1.0,19.0,1964.0
3,2005.0,1.0,19.0,2004.0
4,2005.0,1.0,19.0,1998.0


Basic feature engineering - age at the time of accident

In [20]:
df_num2['user_age'] = df_num2['year'] - df_num2['year_of_birth']
df_num3 = df_num2.drop(['year_of_birth'], axis =1 )

In [21]:
df_num_out = df_num3.copy()

In [22]:
df_num_out.head()

,year,month,hour,user_age
0,2005.0,1.0,19.0,29.0
1,2005.0,1.0,19.0,37.0
2,2005.0,1.0,19.0,41.0
3,2005.0,1.0,19.0,1.0
4,2005.0,1.0,19.0,7.0


### Merging categorical and numeric columns

In [23]:
df_out = df_cat_out.join(df_num_out)

In [24]:
df_out['y'] = master_table.y

In [25]:
df_out.head()

,d_lighting_Night with public lighting not lit,d_lighting_Night with public lighting on,d_lighting_Night without public lighting,d_lighting_Twilight or dawn,d_localization_Out of agglomeration,d_intersection_type_Intersection in T,d_intersection_type_Intersection in X,d_intersection_type_Intersection in Y,d_intersection_type_Intersection with more than 4 branches,d_intersection_type_Level crossing,d_intersection_type_NA,d_intersection_type_Other intersection,d_intersection_type_Out of intersection,d_intersection_type_Place,d_weather_Dazzling weather,d_weather_Fog - smoke,d_weather_Heavy rain,d_weather_Light rain,d_weather_NA,d_weather_Normal,d_weather_Other,d_weather_Snow - hail,d_weather_Strong wind - storm,d_collision_type_Other collision,d_collision_type_Three or more vehicles - multiple collisions,d_collision_type_Three vehicles and more - in chain,d_collision_type_Two vehicles - by the side,d_collision_type_Two vehicles - from the rear,d_collision_type_Two vehicles - frontal,d_collision_type_Without collision,d_road_category_Departmental Road,d_road_category_Highway,d_road_category_NA,d_road_category_National Road,d_road_category_Off public network,d_road_category_Parking lot open to public traffic,d_road_category_other,d_road_regime_NA,d_road_regime_One way,d_road_regime_Separated carriageways,d_road_regime_With variable assignment channels,d_reserved_lane_Cycle Bank,d_reserved_lane_NA,d_reserved_lane_Reserved channel,d_road_gradient_Hill bottom,d_road_gradient_Hilltop,d_road_gradient_NA,d_road_gradient_Slope,d_road_plan_Curved right,"d_road_plan_In ""S""",d_road_plan_NA,d_road_plan_Straight part,d_road_condition_fat - oil,d_road_condition_flooded,d_road_condition_icy,d_road_condition_mud,d_road_condition_normal,d_road_condition_other,d_road_condition_puddles,d_road_condition_snow,d_road_condition_wet,d_infrastructure_Carrefour arranged,d_infrastructure_Exchanger or connection brace,d_infrastructure_NA,d_infrastructure_Pedestrian area,d_infrastructure_Railway,d_infrastructure_Toll zone,d_infrastructure_Underground - tunnel,d_accident_situation_On bike path,d_accident_situation_On emergency stop band,d_accident_situation_On the road,d_accident_situation_On the sidewalk,d_accident_situation_On the verge,d_place_in_car_1,d_place_in_car_2,d_place_in_car_3,d_place_in_car_4,d_place_in_car_5,d_place_in_car_6,d_place_in_car_7,d_place_in_car_8,d_place_in_car_9,d_place_in_car_NA,d_user_type_Passenger,d_user_type_Pedestrian,d_user_type_Pedestrian in rollerblade or scooter,d_sex_Male,d_equipment_used_no,d_equipment_used_yes,d_pedestrian_action_Masked,d_pedestrian_action_Meaning bumping vehicle,d_pedestrian_action_NA,d_pedestrian_action_Opposite direction of the vehicle,d_pedestrian_action_Other,d_pedestrian_action_Playing - running,d_pedestrian_action_With animal,d_pedestrian_action_not specified or not applicable,d_pedestrian_alone_In a group,d_pedestrian_alone_NA,d_pedestrian_alone_Only,year,month,hour,user_age,y
0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,2005.0,1.0,19.0,29.0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,2005.0,1.0,19.0,37.0,1
2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,2005.0,1.0,19.0,41.0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,2005.0,1.0,19.0,1.0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,

## Feature selection - removing near-zero-variance predictors

In [26]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE, VarianceThreshold

In [27]:
df_num_out.var()

year         11.797123
month        11.273194
hour         21.691832
user_age    330.088458
dtype: float64

Numerical variables are not near-zero-variance

In [28]:
variances = []

for col in df_out.columns:
    variances.append(df_out[col].var())

In [29]:
pd.DataFrame({'col': list(df_out.columns), 'var': variances}).sort_values(by = 'var').head(10)

,col,var
32,d_road_category_NA,0.000001
18,d_weather_NA,0.000062
10,d_intersection_type_NA,0.000135
95,d_pedestrian_action_With animal,0.000185
53,d_road_condition_flooded,0.000385
55,d_road_condition_mud,0.000437
81,d_place_in_car_9,0.000671
66,d_infrastructure_Toll zone,0.000828
78,d_place_in_car_6,0.000863
91,d_pedestrian_action_NA,0.000944


For categorical variables - some have very low variance. A better way to filter these out is probably remove variables with very low percentage.

In [31]:
df_out.select_dtypes('uint8').apply(lambda x: np.mean(x==1)).sort_values()

d_road_category_NA                                     0.000001
d_weather_NA                                           0.000062
d_intersection_type_NA                                 0.000135
d_pedestrian_action_With animal                        0.000185
d_road_condition_flooded                               0.000385
                                                         ...   
d_accident_situation_On the road                       0.881517
d_infrastructure_NA                                    0.887186
d_pedestrian_action_not specified or not applicable    0.918675
d_pedestrian_alone_NA                                  0.920466
d_reserved_lane_NA                                     0.939200
Length: 100, dtype: float64

In [32]:
perc_tre = 0.02

one_perc = df_out.select_dtypes('uint8').apply(lambda x: np.mean(x==1))
non_zero_var_cols = list(one_perc[one_perc > perc_tre][one_perc < 1 - perc_tre].index)

In [33]:
non_zero_var_cols = non_zero_var_cols + list(df_num_out.columns)

In [34]:
df_out2 = df_out[non_zero_var_cols + ['y']]

In [35]:
df_out2.head()

,d_lighting_Night with public lighting on,d_lighting_Night without public lighting,d_lighting_Twilight or dawn,d_localization_Out of agglomeration,d_intersection_type_Intersection in T,d_intersection_type_Intersection in X,d_intersection_type_Out of intersection,d_weather_Heavy rain,d_weather_Light rain,d_weather_Normal,d_collision_type_Other collision,d_collision_type_Three or more vehicles - multiple collisions,d_collision_type_Three vehicles and more - in chain,d_collision_type_Two vehicles - by the side,d_collision_type_Two vehicles - from the rear,d_collision_type_Two vehicles - frontal,d_collision_type_Without collision,d_road_category_Departmental Road,d_road_category_Highway,d_road_category_National Road,d_road_regime_NA,d_road_regime_One way,d_road_regime_Separated carriageways,d_reserved_lane_NA,d_reserved_lane_Reserved channel,d_road_gradient_NA,d_road_gradient_Slope,d_road_plan_Curved right,d_road_plan_NA,d_road_plan_Straight part,d_road_condition_normal,d_road_condition_wet,d_infrastructure_Carrefour arranged,d_infrastructure_NA,d_accident_situation_On the road,d_accident_situation_On the verge,d_place_in_car_1,d_place_in_car_2,d_place_in_car_3,d_place_in_car_4,d_place_in_car_NA,d_user_type_Passenger,d_user_type_Pedestrian,d_sex_Male,d_equipment_used_no,d_equipment_used_yes,d_pedestrian_action_not specified or not applicable,d_pedestrian_alone_NA,d_pedestrian_alone_Only,year,month,hour,user_age,y
0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,19.0,29.0,0
1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,2005.0,1.0,19.0,37.0,1
2,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,1,1,0,2005.0,1.0,19.0,41.0,0
3,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,1,1,0,2005.0,1.0,19.0,1.0,0
4,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,1,1,1,0,2005.0,1.0,19.0,7.0,0


## Feature selection - mutual information

In [36]:
minfos_cat = []

In [37]:
for col in df_out2.select_dtypes('uint8').columns:
#     print(col)
    minfos_cat.append(mutual_info_classif(df_out2[col].values.reshape(-1,1),
                                                        df_out2["y"].values, discrete_features = True)[0])

In [38]:
minfos2= [(col, val) for col, val in zip(df_out2.select_dtypes('uint8').columns, minfos_cat)]

In [39]:
pd.DataFrame(minfos2, columns = ['column', 'mutual_info']).sort_values(by='mutual_info', ascending=False).head()

,column,mutual_info
17,d_road_category_Departmental Road,0.018954
3,d_localization_Out of agglomeration,0.018480
35,d_accident_situation_On the verge,0.012684
34,d_accident_situation_On the road,0.007342
42,d_user_type_Pedestrian,0.006981


Don't run below - takes forever and brakes everything

In [39]:
# minfos_num = []
# for col in df_out2.select_dtypes('float64').columns:
#     print(col)
#     minfos_num.append(mutual_info_regression(df_out2[col].values.reshape(-1,1),
#                                                         df_out2["y"].values))

## Train test split

For each accident in the data, there can be multiple users engaged. This means that we should control for data leakage. People from one accident should be placed together in either train or test dataset. One way to achieve this and simultanously keep distribution of y is to sample accident_id, and from that obtain train test split.


In [40]:
master_table['accident_id']

0          200500000001
1          200500000001
2          200500000001
3          200500000001
4          200500000001
               ...     
1876000    201600059431
1876001    201600059431
1876002    201600059432
1876003    201600059432
1876004    201600059432
Name: accident_id, Length: 1876005, dtype: object

In [42]:
df_out2['accident_id'] = master_table['accident_id']#.groupby('y').accident_id.count()

Test what proportion of y we need in both train and test datasets

In [43]:
df_out2.y.value_counts(normalize = True)

0    0.763189
1    0.236811
Name: y, dtype: float64

Around 23% of y is equal to 1 (seriously injured or killed).
Randomly select 30% of accident_id and check the distribution of y obtained:

In [44]:
temp = pd.DataFrame({'test_id': df_out2.accident_id.sample(frac = 0.3), 'if_test': 1}).set_index('test_id')

temp2 = df_out2.set_index('accident_id').join(temp)
temp3 = temp2.reset_index().rename({'index': 'accident_id'}, axis = 1)
temp3.head()

,accident_id,d_lighting_Night with public lighting on,d_lighting_Night without public lighting,d_lighting_Twilight or dawn,d_localization_Out of agglomeration,d_intersection_type_Intersection in T,d_intersection_type_Intersection in X,d_intersection_type_Out of intersection,d_weather_Heavy rain,d_weather_Light rain,d_weather_Normal,d_collision_type_Other collision,d_collision_type_Three or more vehicles - multiple collisions,d_collision_type_Three vehicles and more - in chain,d_collision_type_Two vehicles - by the side,d_collision_type_Two vehicles - from the rear,d_collision_type_Two vehicles - frontal,d_collision_type_Without collision,d_road_category_Departmental Road,d_road_category_Highway,d_road_category_National Road,d_road_regime_NA,d_road_regime_One way,d_road_regime_Separated carriageways,d_reserved_lane_NA,d_reserved_lane_Reserved channel,d_road_gradient_NA,d_road_gradient_Slope,d_road_plan_Curved right,d_road_plan_NA,d_road_plan_Straight part,d_road_condition_normal,d_road_condition_wet,d_infrastructure_Carrefour arranged,d_infrastructure_NA,d_accident_situation_On the road,d_accident_situation_On the verge,d_place_in_car_1,d_place_in_car_2,d_place_in_car_3,d_place_in_car_4,d_place_in_car_NA,d_user_type_Passenger,d_user_type_Pedestrian,d_sex_Male,d_equipment_used_no,d_equipment_used_yes,d_pedestrian_action_not specified or not applicable,d_pedestrian_alone_NA,d_pedestrian_alone_Only,year,month,hour,user_age,y,if_test
0,200500000001,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,19.0,29.0,0,1.0
1,200500000001,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,19.0,29.0,0,1.0
2,200500000001,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,19.0,29.0,0,1.0
3,200500000001,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,2005.0,1.0,19.0,37.0,1,1.0
4,200500000001,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,2005.0,1.0,19.0,37.0,1,1.0


Finally, create training and test sets

In [45]:
test = temp3.query('if_test == 1').drop('if_test', axis =1)
train = temp3[temp3.if_test.isna()].drop('if_test', axis =1)

Checking if procedure was correct:

In [68]:
print(test.shape)
print(train.shape)

print(test.y.mean())
print(train.y.mean())

(1604704, 55)
(777872, 55)
0.21691103156719246
0.25472699878643273


In [46]:
x_train = train.drop(['y', 'accident_id'], axis = 1)
x_test = test.drop(['y', 'accident_id'], axis = 1)

y_train = train[['y']]
y_test = test[['y']]

In [47]:
x_train.head()

,d_lighting_Night with public lighting on,d_lighting_Night without public lighting,d_lighting_Twilight or dawn,d_localization_Out of agglomeration,d_intersection_type_Intersection in T,d_intersection_type_Intersection in X,d_intersection_type_Out of intersection,d_weather_Heavy rain,d_weather_Light rain,d_weather_Normal,d_collision_type_Other collision,d_collision_type_Three or more vehicles - multiple collisions,d_collision_type_Three vehicles and more - in chain,d_collision_type_Two vehicles - by the side,d_collision_type_Two vehicles - from the rear,d_collision_type_Two vehicles - frontal,d_collision_type_Without collision,d_road_category_Departmental Road,d_road_category_Highway,d_road_category_National Road,d_road_regime_NA,d_road_regime_One way,d_road_regime_Separated carriageways,d_reserved_lane_NA,d_reserved_lane_Reserved channel,d_road_gradient_NA,d_road_gradient_Slope,d_road_plan_Curved right,d_road_plan_NA,d_road_plan_Straight part,d_road_condition_normal,d_road_condition_wet,d_infrastructure_Carrefour arranged,d_infrastructure_NA,d_accident_situation_On the road,d_accident_situation_On the verge,d_place_in_car_1,d_place_in_car_2,d_place_in_car_3,d_place_in_car_4,d_place_in_car_NA,d_user_type_Passenger,d_user_type_Pedestrian,d_sex_Male,d_equipment_used_no,d_equipment_used_yes,d_pedestrian_action_not specified or not applicable,d_pedestrian_alone_NA,d_pedestrian_alone_Only,year,month,hour,user_age
20,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,18.0,22.0
21,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,18.0,49.0
22,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,16.0,85.0
23,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,1,1,1,0,2005.0,1.0,16.0,82.0
24,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,2005.0,1.0,16.0,41.0


In [48]:
# x_train, x_test, y_train, y_test = train_test_split(df_out2.drop('y', axis=1), 
#                                                     df_out2.y, 
#                                                     stratify=df_out2.y, 
#                                                     test_size=0.3, 
#                                                     random_state = 3)

### Modeling

In [48]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
cols_to_log = [
#  'd_lighting_Night with public lighting on',
#  'd_lighting_Night without public lighting',
#  'd_lighting_Twilight or dawn',
#  'd_localization_Out of agglomeration',
#  'd_intersection_type_Intersection in T',
#  'd_intersection_type_Intersection in X',
#  'd_intersection_type_Out of intersection',
#  'd_weather_Heavy rain',
#  'd_weather_Light rain',
#  'd_weather_Normal',
#  'd_collision_type_Other collision',
#  'd_collision_type_Three or more vehicles - multiple collisions',
#  'd_collision_type_Three vehicles and more - in chain',
#  'd_collision_type_Two vehicles - by the side',
#  'd_collision_type_Two vehicles - from the rear',
#  'd_collision_type_Two vehicles - frontal',
#  'd_collision_type_Without collision',
 'd_road_category_Departmental Road',
#  'd_road_category_Highway',
#  'd_road_category_National Road',
#  'd_road_regime_NA',
#  'd_road_regime_One way',
#  'd_road_regime_Separated carriageways',
#  'd_reserved_lane_NA',
#  'd_reserved_lane_Reserved channel',
#  'd_road_gradient_NA',
#  'd_road_gradient_Slope',
#  'd_road_plan_Curved right',
#  'd_road_plan_NA',
#  'd_road_plan_Straight part',
#  'd_road_condition_normal',
#  'd_road_condition_wet',
#  'd_infrastructure_Carrefour arranged',
#  'd_infrastructure_NA',
#  'd_accident_situation_On the road',
#  'd_accident_situation_On the verge',
#  'd_place_in_car_1',
#  'd_place_in_car_2',
#  'd_place_in_car_3',
#  'd_place_in_car_4',
#  'd_place_in_car_NA',
#  'd_user_type_Passenger',
#  'd_user_type_Pedestrian',
#  'd_sex_Male',
#  'd_equipment_used_no',
#  'd_equipment_used_yes',
#  'd_pedestrian_action_not specified or not applicable',
#  'd_pedestrian_alone_NA',
#  'd_pedestrian_alone_Only',
#  'year',
#  'month',
#  'hour',
#  'user_age'
 ]


mod = sm.GLM(y_train['y'],x_train[cols_to_log], family=sm.families.Binomial())
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:               777422
Model:                            GLM   Df Residuals:                   777421
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -5.3375e+05
Date:                Sun, 17 May 2020   Deviance:                   1.0675e+06
Time:                        17:09:55   Pearson chi2:                 7.77e+05
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
d_road_category_Departmental Road    -0.4178      0.004   -100.053      0.000      -0.426      -0.410
=====================================================================================================
"""

We are following the mutual info result for building GLM.

In [49]:
cols_to_log = [
#  'd_lighting_Night with public lighting on',
#  'd_lighting_Night without public lighting',
#  'd_lighting_Twilight or dawn',
 'd_localization_Out of agglomeration',
#  'd_intersection_type_Intersection in T',
#  'd_intersection_type_Intersection in X',
#  'd_intersection_type_Out of intersection',
#  'd_weather_Heavy rain',
#  'd_weather_Light rain',
#  'd_weather_Normal',
#  'd_collision_type_Other collision',
#  'd_collision_type_Three or more vehicles - multiple collisions',
#  'd_collision_type_Three vehicles and more - in chain',
#  'd_collision_type_Two vehicles - by the side',
#  'd_collision_type_Two vehicles - from the rear',
#  'd_collision_type_Two vehicles - frontal',
#  'd_collision_type_Without collision',
 'd_road_category_Departmental Road',
#  'd_road_category_Highway',
#  'd_road_category_National Road',
#  'd_road_regime_NA',
#  'd_road_regime_One way',
#  'd_road_regime_Separated carriageways',
#  'd_reserved_lane_NA',
#  'd_reserved_lane_Reserved channel',
#  'd_road_gradient_NA',
#  'd_road_gradient_Slope',
#  'd_road_plan_Curved right',
#  'd_road_plan_NA',
#  'd_road_plan_Straight part',
#  'd_road_condition_normal',
#  'd_road_condition_wet',
#  'd_infrastructure_Carrefour arranged',
#  'd_infrastructure_NA',
#  'd_accident_situation_On the road',
#  'd_accident_situation_On the verge',
#  'd_place_in_car_1',
#  'd_place_in_car_2',
#  'd_place_in_car_3',
#  'd_place_in_car_4',
#  'd_place_in_car_NA',
#  'd_user_type_Passenger',
#  'd_user_type_Pedestrian',
#  'd_sex_Male',
#  'd_equipment_used_no',
#  'd_equipment_used_yes',
#  'd_pedestrian_action_not specified or not applicable',
#  'd_pedestrian_alone_NA',
#  'd_pedestrian_alone_Only',
#  'year',
#  'month',
#  'hour',
#  'user_age'
 ]
mod = sm.GLM(y_train['y'],x_train[cols_to_log],family = sm.families.Binomial())
res= mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:               777422
Model:                            GLM   Df Residuals:                   777420
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -5.3264e+05
Date:                Sun, 17 May 2020   Deviance:                   1.0653e+06
Time:                        17:10:06   Pearson chi2:                 7.78e+05
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
d_localization_Out of agglomeration    -0.2309      0.005    -47.047      0.000      -0.241      -0.221
d_road_category_Departmental Road      -0.2988      0.005    -61.316      0.000      -0.308      -0.289
=======================================================================================================
"""

In [57]:
cols_to_log = [
#  'd_lighting_Night with public lighting on',
#  'd_lighting_Night without public lighting',
#  'd_lighting_Twilight or dawn',
 'd_localization_Out of agglomeration',
#  'd_intersection_type_Intersection in T',
#  'd_intersection_type_Intersection in X',
#  'd_intersection_type_Out of intersection',
#  'd_weather_Heavy rain',
#  'd_weather_Light rain',
#  'd_weather_Normal',
#  'd_collision_type_Other collision',
#  'd_collision_type_Three or more vehicles - multiple collisions',
#  'd_collision_type_Three vehicles and more - in chain',
#  'd_collision_type_Two vehicles - by the side',
#  'd_collision_type_Two vehicles - from the rear',
#  'd_collision_type_Two vehicles - frontal',
#  'd_collision_type_Without collision',
 'd_road_category_Departmental Road',
#  'd_road_category_Highway',
#  'd_road_category_National Road',
#  'd_road_regime_NA',
#  'd_road_regime_One way',
#  'd_road_regime_Separated carriageways',
#  'd_reserved_lane_NA',
#  'd_reserved_lane_Reserved channel',
#  'd_road_gradient_NA',
#  'd_road_gradient_Slope',
#  'd_road_plan_Curved right',
#  'd_road_plan_NA',
#  'd_road_plan_Straight part',
#  'd_road_condition_normal',
#  'd_road_condition_wet',
#  'd_infrastructure_Carrefour arranged',
#  'd_infrastructure_NA',
 'd_accident_situation_On the road',
 'd_accident_situation_On the verge',
#  'd_place_in_car_1',
#  'd_place_in_car_2',
#  'd_place_in_car_3',
#  'd_place_in_car_4',
#  'd_place_in_car_NA',
#  'd_user_type_Passenger',
 'd_user_type_Pedestrian'
#  'd_sex_Male',
#  'd_equipment_used_no',
#  'd_equipment_used_yes',
#  'd_pedestrian_action_not specified or not applicable',
#  'd_pedestrian_alone_NA',
#  'd_pedestrian_alone_Only',
#  'year',
#  'month',
#  'hour',
#  'user_age'
 ]
mod = sm.GLM(y_train['y'],x_train[cols_to_log],family = sm.families.Binomial())
res= mod.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:               777422
Model:                            GLM   Df Residuals:                   777417
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.0902e+05
Date:                Sun, 17 May 2020   Deviance:                   8.1805e+05
Time:                        17:16:15   Pearson chi2:                 7.69e+05
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
d_localization_Out of agglomeration     0.7053      0.006    115.611      0.000       0.693       0.717
d_road_category_Departmental Road       0.6517      0.006    110.214      0.000       0.640       0.663
d_accident_situation_On the road       -1.7867      0.004   -415.285      0.000      -1.795      -1.778
d_accident_situation_On the verge      -0.3614      0.012    -30.074      0.000      -0.385      -0.338
d_user_type_Pedestrian                  1.0749      0.009    125.469      0.000       1.058       1.092
=======================================================================================================
"""

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')
%matplotlib inline
cols_to_log = [
 'd_lighting_Night with public lighting on',
 'd_lighting_Night without public lighting',
 'd_lighting_Twilight or dawn',
 'd_localization_Out of agglomeration',
 'd_intersection_type_Intersection in T',
 'd_intersection_type_Intersection in X',
 'd_intersection_type_Out of intersection',
 'd_weather_Heavy rain',
 'd_weather_Light rain',
 'd_weather_Normal',
 'd_collision_type_Other collision',
 'd_collision_type_Three or more vehicles - multiple collisions',
 'd_collision_type_Three vehicles and more - in chain',
 'd_collision_type_Two vehicles - by the side',
 'd_collision_type_Two vehicles - from the rear',
 'd_collision_type_Two vehicles - frontal',
 'd_collision_type_Without collision',
 'd_road_category_Departmental Road',
 'd_road_category_Highway',
 'd_road_category_National Road',
 'd_road_regime_NA',
 'd_road_regime_One way',
 'd_road_regime_Separated carriageways',
 'd_reserved_lane_NA',
 'd_reserved_lane_Reserved channel',
 'd_road_gradient_NA',
 'd_road_gradient_Slope',
 'd_road_plan_Curved right',
 'd_road_plan_NA',
 'd_road_plan_Straight part',
 'd_road_condition_normal',
 'd_road_condition_wet',
 'd_infrastructure_Carrefour arranged',
 'd_infrastructure_NA',
 'd_accident_situation_On the road',
 'd_accident_situation_On the verge',
 'd_place_in_car_1',
 'd_place_in_car_2',
 'd_place_in_car_3',
 'd_place_in_car_4',
 'd_place_in_car_NA',
 'd_user_type_Passenger',
 'd_user_type_Pedestrian'
 'd_sex_Male',
 'd_equipment_used_no',
 'd_equipment_used_yes',
 'd_pedestrian_action_not specified or not applicable',
 'd_pedestrian_alone_NA',
 'd_pedestrian_alone_Only',
 'year',
 'month',
 'hour',
 'user_age'
 ]
mod = sm.GLM(y_train['y'],x_train[cols_to_log],family = sm.families.Binomial())
res= mod.fit()
res.summary()
probs = res.predict()
fpr, tpr, _ = roc_curve((y_train.y=="0"), probs)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

Dont know why it does not show the plot ? any idea ?

KNN is slooow

In [49]:
# model = KNeighborsClassifier()
# a = cross_validate(model, df_out, y_train, cv=3, scoring = 'roc_auc', verbose = 1)

In [4]:
model1 = LogisticRegression(n_jobs=-1, verbose=2)
a = cross_validate(model1, x_train, y_train, cv=2, scoring = 'roc_auc', verbose = 1, n_jobs=-1)

NameError: name 'x_train' is not defined

In [1]:
fitted = model1.fit(x_train, y_train)

NameError: name 'model1' is not defined

In [48]:
roc_auc_score(y_train, fitted.predict_proba(x_train)[:,1])


0.7423474357220142

## Stepwise feature selection

In [49]:
from sklearn.svm import LinearSVC

In [50]:
model2 = LinearSVC()

In [51]:
model2.fit(x_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [52]:
from sklearn.feature_selection import RFECV, RFE

In [55]:
rfe = RFE(model2, n_features_to_select = 1)

In [ ]:
rfe.fit(x_train, y_train)